# Analysis of MF Data from AMFI and Portfolio Development

This notebook aims to analyze all listed MFs under AMFI (Association of Mutual Funds of India) to provide insights on volatility and historical performance of each fund and leverage the same to suggest recommendations for investment.

In [16]:
# Import neccessary libraries
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Initialize environment
load_dotenv()
amfi_data_batchA = os.getenv('amfi_data_batchA')
amfi_data_batchB = os.getenv('amfi_data_batchB')
railway_db_url = os.getenv('railway_db_url')
engine = create_engine(railway_db_url, connect_args={'options': '-c search_path="FINANCIAL_ANALYSIS"'})

# Select execution option
option = input('Select program to run: 1-Data_Load, 2-Funds_Analysis: ')

## Data load program - to be executed only once in the beginning

In [17]:
# Data load program
if option == '1':
    try:
        # Load data from batch A into dataframe and correct date format
        df_batchA = pd.read_csv(amfi_data_batchA)
        df_batchA['date'] = pd.to_datetime(df_batchA['date'], infer_datetime_format= True, errors = 'coerce')
        print(f'Data for top 5 rows from batch A: \n{df_batchA.head(5)}')
        errors_batchA = df_batchA['date'].isna()
        i_batchA = [rows for rows, val in enumerate(errors_batchA) if val == True]
        print(f'Number of records with dates in string and not updated by Pandas in batch A: {len(i_batchA)}')
        df_batchA = df_batchA.dropna(subset=['date'])
        print(f'Number of records in batch A: {len(df_batchA['SNo.'])}')

        # Load data from batch B into dataframe and correct date format
        df_batchB = pd.read_csv(amfi_data_batchB)
        df_batchB['date'] = pd.to_datetime(df_batchB['date'], infer_datetime_format= True, errors='coerce')
        print(f'Data for top 5 rows from batch B: \n{df_batchB.head(5)}')
        errors_batchB = df_batchB['date'].isna()
        i_batchB = [rows for rows, val in enumerate(errors_batchB) if val == True]
        print(f'Number of records with dates in string and not updated by Pandas in batch B: {len(i_batchB)}')
        df_batchB = df_batchB.dropna(subset=['date'])
        print(f'Number of records in batch B: {len(df_batchB['SNo.'])}')

        # Combine batch A and B data
        df_combined = pd.concat([df_batchA, df_batchB], ignore_index=True)
        df_combined = df_combined.rename(columns={'date': 'trx_date'})
        df_combined = df_combined.rename(columns={'SNo.': 's_no'})
        print(f'Data for top 5 rows from consolidated dataframe: \n{df_combined.head(5)}')
        print(f'Number of records in consolidated data: {len(df_combined['s_no'])}')

        # Populate data into database
        with engine.connect() as database_connection:
            for records_start in range(0, len(df_combined), 1000000):
                records_end = records_start + 1000000
                df_chunks = df_combined.iloc[records_start:records_end]
                df_chunks.to_sql(
                    'amfi_database',
                    con=database_connection,
                    schema='FINANCIAL_ANALYSIS',
                    if_exists='append',
                    index=False,
                    method='multi'
                    )
                database_connection.commit()
                print(f'{len(df_chunks)} Committed.')

        query = text('select * from amfi_database;')
        with engine.connect() as database_connection:
            df = pd.read_sql(sql=query, con=database_connection, index_col='trx_id')

        processed_records = len(df['s_no'])
        print(f'Successfully entered {processed_records} into database.')
    except Exception as e:
        print(f'Error: {e}')
else:
    print(f'Selected option 2. Proceeding to execute funds analysis program.')

Selected option 2. Proceeding to execute funds analysis program.


## Funds analysis program

### Data pull and initial analysis to record volatility and additional information using GPU enabled Data Frame (RAPIDS cuDF)

In [18]:
# Funds analysis program - data pull and load
if option == '2':
    import cudf
    import cupy as cp
    import connectorx as cx
    
    try:
        # Pull data into arrow table using connectorx and convert to GPU enabled dataframe.
        query = 'select * from "FINANCIAL_ANALYSIS".amfi_database where trading_symbol_reinvestment is not null and trading_symbol_growth is not null;'
        nav_historical_arrow_tbl = cx.read_sql(
            conn=railway_db_url, 
            query=query, 
            return_type='arrow',
            partition_on='trx_id',
            partition_range=(1, 20000000),
            partition_num=7
            )
        df_nav_historical_data = cudf.DataFrame.from_arrow(nav_historical_arrow_tbl)
        df_nav_historical_data = df_nav_historical_data.sort_values(by='s_no')
        print(f'MFs with available trading symbols:{len(df_nav_historical_data)}')
        df_nav_historical_data_20_year = df_nav_historical_data[
                df_nav_historical_data['trx_date'].dt.year>=2005
            ]
        print(f'20 year historical records: {len(df_nav_historical_data_20_year)}')

    except Exception as e:
        print(f'Error Encountered: {e}')

elif option == '1':
    print('Selected option 1, running the data load program.')
else:
    print(f'Invalid option {option} selected. Please enter either 1 or 2 as input.')

MFs with available trading symbols:4702825
20 year historical records: 4702825
